# Task1. Spam or Ham text classifition

## Sergei Bruchkus

In [207]:
#to begin with, we import necessary modules
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
%matplotlib inline 

#import nltk
#from nltk.corpus import stopwords
#import re
#import string
#from nltk.corpus import stopwords
#from collections import Counter


from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vector = CountVectorizer()
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer

In [208]:
#upload the data from smsspam csv file
df = pd.read_csv('smsspam.csv',usecols = [0,1],encoding='latin-1' )
df.rename(columns = {'v1':'Label','v2': 'Message'}, inplace = True)

In [209]:
#check how the data looks like
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [210]:
#for a more detailed understanding about each category
df.groupby('Label').describe()

Message                                                               
        count unique                                                top freq
Label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

Here we can notice small imbalance of our sample, due to dominance of ham messages. This is the case when even if naive classificator recognizes most textes as the major class - the overall accuracy still comparetively high.

The idea is to to convert this set of text to a frequency distribution matrix, where we can see directly the frequencies of every word

In [211]:
#firstly, using CountVectorizer we adjust the text, bringing it to lowercase, removing punctuation and stopwords 
from sklearn.feature_extraction.text import CountVectorizer
count_vector= CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [212]:
X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['Label'],test_size=0.2)
print('Training set: {}'.format(X_train.shape[0]))
print('Test set: {}'.format(X_test.shape[0]))

Training set: 4457
Test set: 1115


In [213]:
count_vector.fit(X_train)
count_vector.get_feature_names()

['00',
 '000',
 '008704050406',
 '0089',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '050703',
 '0578',
 '06',
 '07',
 '07008009200',
 '07090201529',
 '07090298926',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07781482378',
 '07786200117',
 '077xxx',
 '078',
 '07808247860',
 '07808726822',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '08',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '083',
 '0844',
 '08448714184',
 '0845',
 '08450542832',
 '08452810071',
 '08452810073',
 '08452810075over18',
 '0870',
 '08700435505150p',
 '08700469649',
 '08700621170150

In [214]:
cv_array=count_vector.transform(X_train).toarray()
cv_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [215]:
freqmat=pd.DataFrame(cv_array, columns=count_vector.get_feature_names())
freqmat[:100]
#actually it was expected to be more informative (but because of the giant sample size we can not see tha details)

,00,000,008704050406,0089,0121,01223585236,01223585334,0125698789,02,0207,...,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,¹ð,ðµð
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
#We create sub-dataframe of ham textes
ham_1 = df[df['Label'] == 'ham'] 

#We create sub-dataframe of spam textes
spam_1 = df[df['Label'] == 'spam'] 

In [217]:
ham_1
#too much slang...

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...
...,...,...
5565,ham,Huh y lei...
5568,ham,Will Ð_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


We will start with spam messages, since it is the class of our interest

In [218]:
spam_1

,Label,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [219]:
spam_1['Message']

2       Free entry in 2 a wkly comp to win FA Cup fina...
5       FreeMsg Hey there darling it's been 3 week's n...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
11      SIX chances to win CASH! From 100 to 20,000 po...
                              ...                        
5537    Want explicit SEX in 30 secs? Ring 02073162414...
5540    ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547    Had your contract mobile 11 Mnths? Latest Moto...
5566    REMINDER FROM O2: To get 2.50 pounds free call...
5567    This is the 2nd time we have tried 2 contact u...
Name: Message, Length: 747, dtype: object

In [220]:
textspam=spam_1['Message'].tolist()
count_vector.fit(textspam)
count_vector.get_feature_names()

['00',
 '000',
 '008704050406',
 '0089',
 '0121',
 '01223585236',
 '01223585334',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '050703',
 '0578',
 '06',
 '07',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07781482378',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '08',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '083',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08450542832',
 '08452810071',
 '08452810073',

In [221]:

# Python program to convert a list 
# to string using join() function 
    
# Function to convert   
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 


In [223]:
s1=listToString(textspam)
#s1

#узнать общее число спамовых слов
s2=s1.split()

l = len(s2)

print(l)

17817


Now we are going to check the frequency of particular words in spam messages

In [224]:
sp=count_vector.fit_transform(textspam)
#sp.toarray()


#узнаем количественное вхождение каждого слова:
matrix_freq = np.asarray(sp.sum(axis=0)).ravel()
final_matrix = np.array([np.array(count_vector.get_feature_names()), matrix_freq])
final_matrix=final_matrix.transpose() #трансопнируем, чтобы было нагляднее
#final_matrix.to
#final_matrix_sorted = sorted(final_matrix.values, key=lambda item: item[1]) 

print(final_matrix)

[['00' '10']
 ['000' '29']
 ['008704050406' '2']
 ...
 ['zoe' '1']
 ['zouk' '1']
 ['ðµð' '301']]


In [225]:
#попытка сделать таблицу, где в первом столбце - слова из спамовых сообщений, во втором - частота
final_table_spam=pd.DataFrame(final_matrix)
final_table_spam.columns = ['word','frequency']
final_table_spam

,word,frequency
0,00,10
1,000,29
2,008704050406,2
3,0089,1
4,0121,1
...,...,...
2858,zebra,1
2859,zed,6
2860,zoe,1
2861,zouk,1


Other sort methods didn't work out. Even though the method I used somehow got rid of several features (the last one with frequency 301, for example).

In [226]:
#range by the frequency of appearing

final_table_spam1=final_table_spam.sort_values(by=['frequency'], ascending=False 
                            )
final_table_spam1

,word,frequency
2821,www,98
1946,of,95
2109,prize,93
876,best,9
2764,weeks,9
...,...,...
1155,darkest,1
1973,or2optout,1
1157,dartboard,1
179,08719899229,1


As far as I get, here we need not a frequency but more like a probability that a certain word can appear in some SPAM text
We can apply Zif law here (?)... the rank in the table=the place in the standing 

Generally we have 2863 unique words that can be met in the spam messages, but we 17817 words geenrally.
Logically, the probability to meet, for example word "www" in spam messages is 98/17817=0,0055...
the same for prize= 93/17817=0,005219...

Now we have to be looking at all of any message’s features individually. 
Bayes Rule below shows us how to calculate the posterior probability for just one feature. 
The posterior probability of the object is calculated for each feature and then these probabilities are multiplied together to get a final probability. This probability is calculated for the other class as well. Which ever has the greater probability that ultimately determines what class the object is in.

P(A/B)=P(B/A)*P(A)/P(B)
here P(A/B) is posterior, 


P(SPAM/Word) = P(Word/SPAM)*P(Spam)/P(Word)
P(Word)=P(Word/SPAM)*P(Spam) + P(Word/Not SPAM)*P(Not Spam) 
Then we combine these probabilities to each word inn the message and decide is it spam or not
P(Spam)=747/(4825+747)=0,1341, P(Not-Spam) is 0,8659

I'm not sure to which extent we should code it with hands
But it's a bit hard to calculate everything...

How to get the accuracy of these predicitions: to estimate if each text is recognized as spam or ham and compare with real label => will get recalls, f1-scores and overall accuracy

In [241]:
#more detailed data about ham, for calculations

textham=ham_1['Message'].tolist()
count_vector.fit(textham)
count_vector.get_feature_names() 

s3=listToString(textham)
#узнать общее число спамовых слов
s4=s3.split()
l1 = len(s4)

print(l)

hamt=count_vector.fit_transform(textham)


#узнаем количественное вхождение каждого слова:
matrix_freq_h = np.asarray(hamt.sum(axis=0)).ravel()
final_matrix_h = np.array([np.array(count_vector.get_feature_names()), matrix_freq_h])
final_matrix_h=final_matrix_h.transpose() #трансопнируем, чтобы было нагляднее 

print(final_matrix_h)

#попытка сделать таблицу, где в первом столбце - слова из спамовых сообщений, во втором - частота
final_table_ham=pd.DataFrame(final_matrix_h)
final_table_ham.columns = ['word','frequency']


#range by the frequency of appearing

final_table_ham1=final_table_ham.sort_values(by=['frequency'], ascending=False 
                            )
final_table_ham1

17817
[['000pes' '1']
 ['0125698789' '1']
 ['0quit' '1']
 ...
 ['zyada' '1']
 ['¹ð' '1']
 ['ðµð' '10']]


,word,frequency
6708,work,97
2491,give,93
361,an,92
5137,say,92
326,already,90
...,...,...
2180,feng,1
2177,fellow,1
2172,feellikw,1
2170,feelin,1


 ## Sklearn Naive Bayes

Solution using sklearn

In [242]:
data = pd.read_csv('smsspam.csv',usecols = [0,1],encoding='latin-1' )
data.rename(columns = {'v1':'Label','v2': 'Message'}, inplace = True)

In [243]:
data.head(5)

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [244]:
X_train, X_test, y_train, y_test = train_test_split(data['Message'],data['Label'],test_size=0.2)

In [245]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer='word')), # strings to int counts
    ('tfidf',TfidfTransformer()), # int counts to weighted TF-IDF scores
    ('classifier',MultinomialNB()) # train on TF-IDF vectors with Naive Bayes classifier
])

In [246]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [247]:
predictions=pipeline.predict(X_test)

In [248]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       978
        spam       1.00      0.72      0.83       137

    accuracy                           0.97      1115
   macro avg       0.98      0.86      0.91      1115
weighted avg       0.97      0.97      0.96      1115



 Overall accuracy is 97%, spam recall is not that high, meaning that a better classifeir could be built. 